# AR6 calibration of FaIR 2.1 using RCP emissions

Calibration
- use the exact aerosol indirect forcing relationship from Smith et al. (2021). 
- aerosol direct is from AR6.
- three layer model for climate response.
- include overlap of the major GHGs.
- prognostic equation for land use related forcing (e.g. from FaIR 1.6).
- ozone relationship from FaIR 1.6 used in AR6.
- interactive methane lifetime (NEW!)

TODO:
- solar trend
- ozone depleting substances should not include HFCs

## Basic imports

In [ ]:
import copy

from climateforcing.utils import mkdir_p
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import time
import scipy.stats
from tqdm.autonotebook import tqdm

from fair21 import (
    SpeciesID, Category, Config, Species, RunMode, Scenario, ClimateResponse, RunConfig, CH4LifetimeMethod, FAIR
)
from fair21.defaults import species_config_from_default
from fair21.energy_balance_model import EnergyBalanceModel

In [ ]:
mkdir_p('../data/ar6_ensemble_batches_rcp/')

## Set up problem

In [ ]:
species_ids = {
    # Greenhouse gases and precursors
    'CO2_FFI': SpeciesID('CO2 fossil fuel and industrial', Category.CO2_FFI),
    'CO2_AFOLU': SpeciesID('CO2 AFOLU', Category.CO2_AFOLU),
    'CO2': SpeciesID('CO2', Category.CO2),
    'CH4': SpeciesID('CH4', Category.CH4),
    'N2O': SpeciesID('N2O', Category.N2O),
    'CFC-11': SpeciesID('CFC-11', Category.CFC_11),
    'CFC-12': SpeciesID('CFC-12', Category.OTHER_HALOGEN),
    'CFC-113': SpeciesID('CFC-113', Category.OTHER_HALOGEN),
    'CFC-114': SpeciesID('CFC-114', Category.OTHER_HALOGEN),
    'CFC-115': SpeciesID('CFC-115', Category.OTHER_HALOGEN),
    'HCFC-22': SpeciesID('HCFC-22', Category.OTHER_HALOGEN),
    'HCFC-141b': SpeciesID('HCFC-141b', Category.OTHER_HALOGEN),
    'HCFC-142b': SpeciesID('HCFC-142b', Category.OTHER_HALOGEN),
    'CCl4': SpeciesID('CCl4', Category.OTHER_HALOGEN),
    'CHCl3': SpeciesID('CHCl3', Category.OTHER_HALOGEN),
    'CH2Cl2': SpeciesID('CH2Cl2', Category.OTHER_HALOGEN),
    'CH3Cl': SpeciesID('CH3Cl', Category.OTHER_HALOGEN),
    'CH3CCl3': SpeciesID('CH3CCl3', Category.OTHER_HALOGEN),
    'CH3Br': SpeciesID('CH3Br', Category.OTHER_HALOGEN),
    'Halon-1211': SpeciesID('Halon-1211', Category.OTHER_HALOGEN),
    'Halon-1301': SpeciesID('Halon-1301', Category.OTHER_HALOGEN),
    'Halon-2402': SpeciesID('Halon-2402', Category.OTHER_HALOGEN),
    'CF4': SpeciesID('CF4', Category.F_GAS),
    'C2F6': SpeciesID('C2F6', Category.F_GAS),
    'C3F8': SpeciesID('C3F8', Category.F_GAS),
    'c-C4F8': SpeciesID('C-C4F8', Category.F_GAS),
    'C4F10': SpeciesID('C4F10', Category.F_GAS),
    'C5F12': SpeciesID('C5F12', Category.F_GAS),
    'C6F14': SpeciesID('C6F14', Category.F_GAS),
    'C7F16': SpeciesID('C7F16', Category.F_GAS),
    'C8F18': SpeciesID('C8F18', Category.F_GAS),
    'HFC-125': SpeciesID('HFC-125', Category.F_GAS),
    'HFC-134a': SpeciesID('HFC-134a', Category.F_GAS),
    'HFC-143a': SpeciesID('HFC-143a', Category.F_GAS),
    'HFC-152a': SpeciesID('HFC-152a', Category.F_GAS),
    'HFC-227ea': SpeciesID('HFC-227ea', Category.F_GAS),
    'HFC-23': SpeciesID('HFC-23', Category.F_GAS),
    'HFC-236fa': SpeciesID('HFC-236fa', Category.F_GAS),
    'HFC-245fa': SpeciesID('HFC-245fa', Category.F_GAS),
    'HFC-32': SpeciesID('HFC-32', Category.F_GAS),
    'HFC-365mfc': SpeciesID('HFC-365mfc', Category.F_GAS),
    'HFC-4310mee': SpeciesID('HFC-4310mee', Category.F_GAS),
    'NF3': SpeciesID('NF3', Category.F_GAS),
    'SF6': SpeciesID('SF6', Category.F_GAS),
    'SO2F2': SpeciesID('SO2F2', Category.F_GAS),
    # aerosols, ozone, and their precursors
    'Sulfur': SpeciesID('Sulfur', Category.SULFUR),
    'BC': SpeciesID('BC', Category.BC),
    'OC': SpeciesID('OC', Category.OC),
    'NH3': SpeciesID('NH3', Category.OTHER_AEROSOL),
    'VOC': SpeciesID('VOC', Category.REACTIVE_GAS),
    'CO': SpeciesID('CO', Category.REACTIVE_GAS),
    'NOx': SpeciesID('NOx', Category.REACTIVE_GAS),
    'ari': SpeciesID('Aerosol-Radiation Interactions', Category.AEROSOL_RADIATION_INTERACTIONS),
    'aci': SpeciesID('Aerosol-Cloud Interactions', Category.AEROSOL_CLOUD_INTERACTIONS),
    'ozone': SpeciesID('Ozone', Category.OZONE),
    # Contrails and precursors
    'NOx_aviation': SpeciesID('NOx Aviation', Category.NOX_AVIATION),
    'contrails': SpeciesID('Contrails', Category.CONTRAILS),
    # other minor anthropogenic
    'LAPSI': SpeciesID('Light absorbing particles on snow and ice', Category.LAPSI),
    'H2O_stratospheric': SpeciesID('H2O Stratospheric', Category.H2O_STRATOSPHERIC),
    'land_use': SpeciesID('Land Use', Category.LAND_USE),
    # natural
    'solar': SpeciesID('Solar', Category.SOLAR),
    'volcanic': SpeciesID('Volcanic', Category.VOLCANIC)
}

In [ ]:
scenarios_to_include=['rcp45']

In [ ]:
df_emis = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
df_conc = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')
df_forc = pd.read_csv('../data/forcing/table_A3.3_historical_ERF_1750-2019_best_estimate.csv')
df_solar = pd.read_csv('../data/forcing/solar_erf.csv')

In [ ]:
emitted_species = [
    'CO2_FFI', 'CO2_AFOLU', 'CH4', 'N2O',
    'Sulfur', 'BC', 'OC', 'NH3', 'NOx', 'VOC', 'CO',
    'CFC-11', 'CFC-12', 'CFC-113', 'CFC-114', 'CFC-115',
    'HCFC-22', 'HCFC-141b', 'HCFC-142b',
    'CCl4', 'CH3Cl', 'CH3CCl3', 'CH3Br',
    'Halon-1211', 'Halon-1301', 'Halon-2402',
    'CF4', 'C2F6', 'C6F14',
    'SF6',
    'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-227ea', 'HFC-23', 'HFC-245fa', 'HFC-32', 
    'HFC-4310mee']
forced_species = ['solar', 'volcanic']
from_other_species = ['CO2', 'ari', 'aci', 'ozone', 'LAPSI', 'H2O_stratospheric', 'land_use']

species_to_include = emitted_species + forced_species + from_other_species

In [ ]:
scenarios = []
for iscen, scenario in enumerate(scenarios_to_include):
    list_of_species = []
    for ispec, species in enumerate(emitted_species):
        species_rcmip_name = species.replace("-", "")
        if species == 'NOx_aviation':
            species_rcmip_name = 'NOx|MAGICC Fossil and Industrial|Aircraft'
        elif species == 'CO2_FFI':
            species_rcmip_name = 'CO2|MAGICC Fossil and Industrial'
        elif species == 'CO2_AFOLU':
            species_rcmip_name = 'CO2|MAGICC AFOLU'
        emis_in = df_emis.loc[
            (df_emis['Scenario']==scenario) & (df_emis['Variable'].str.endswith("|"+species_rcmip_name)) & 
            (df_emis['Region']=='World'), '1765':'2100'
        ].interpolate(axis=1).values.squeeze()

        # CO2 and N2O units need to behave: TODO, sort this out
        if species in ('CO2_FFI', 'CO2_AFOLU', 'N2O'):
            emis_in = emis_in / 1000
        list_of_species.append(Species(species_ids[species], emissions=emis_in))
        #print(species_rcmip_name, emis_in.shape)
        
        # for SLCFs, we will follow what we did in AR6, and use Skeie et al 2011 as baseline emissions
        if species=='BC':
            emis_in[:86] = np.linspace(1.2, emis_in[85], 86)
        if species=='OC':
            emis_in[:86] = np.linspace(10, emis_in[85], 86)
        if species=='CO':
            emis_in[:86] = np.linspace(174, emis_in[85], 86)
        if species=='NH3':
            emis_in[:86] = np.linspace(4, emis_in[85], 86)
        if species=='NOx':
            emis_in[:86] = np.linspace(46/14 * 2, emis_in[85], 86)
        if species=='VOC':
            emis_in[:86] = np.linspace(10, emis_in[85], 86)
        if species=='Sulfur':
            emis_in[:86] = np.linspace(2, emis_in[85], 86)
        
    # solar and volcanic forcing still a little hacky
    # to follow RFF, turn off after 2020
    solar_forcing = np.zeros(336)
    solar_forcing[:255] = df_solar['solar_erf'].loc[1765.5:2020].values
    volcanic_forcing = np.zeros(336)
    volcanic_forcing[:255] = df_forc['volcanic'].values[15:270]
    list_of_species.append(Species(species_ids['solar'], forcing=solar_forcing))
    list_of_species.append(Species(species_ids['volcanic'], forcing=volcanic_forcing))
    
    # add derived species: at this stage just a declaration that we want them
    for species in from_other_species:
        list_of_species.append(Species(species_ids[species]))
        
    scenarios.append(Scenario(scenario, list_of_species))

In [ ]:
samples = 15000
batch_size = 1000

## Load in pre-calculated parameter sets

In [ ]:
df_cc=pd.read_csv('../data/parameter_sets/carbon_cycle.csv')
df_cr=pd.read_csv('../data/parameter_sets/climate_response.csv')
df_aci=pd.read_csv('../data/parameter_sets/erfaci.csv')
df_ari=pd.read_csv('../data/parameter_sets/erfari.csv')
df_ozone=pd.read_csv('../data/parameter_sets/ozone.csv')
df_scaling=pd.read_csv('../data/parameter_sets/forcing_scaling.csv')
df_1750co2=pd.read_csv('../data/parameter_sets/co2_concentration_1750.csv')

In [ ]:
# using an extremely crude curve fit to estimate 1765 CO2

obs_co2 = np.array([278.3, 285.5, 296.4])
obs_years = [1750, 1850, 1900]

def fit(x, a, b, c):
    return a + b*np.exp(c*(x-1750))

p, _ = scipy.optimize.curve_fit(fit, obs_years, obs_co2, p0=[278.3, 1.45, 0.0171])
pl.plot(np.arange(1750, 1900), p[0]+p[1]*np.exp(p[2] * (np.arange(1750, 1900)-1750)))
pl.scatter(obs_years, obs_co2, color='k')

offset1765co2 = fit(1765, p[0], p[1], p[2]) - fit(1750, p[0], p[1], p[2])
offset1765co2

In [ ]:
df_1765co2 = df_1750co2 + offset1765co2

In [ ]:
obs_ch4 = np.array([729.2, 807.6, 925.1])
obs_years = [1750, 1850, 1900]

def fit(x, a, b, c):
    return a + b*np.exp(c*(x-1750))

p, _ = scipy.optimize.curve_fit(fit, obs_years, obs_ch4, p0=[278.3, 1.45, 0.0171])
pl.plot(np.arange(1750, 1900), p[0]+p[1]*np.exp(p[2] * (np.arange(1750, 1900)-1750)))
pl.scatter(obs_years, obs_ch4, color='k')

baseline1765ch4 = fit(1765, p[0], p[1], p[2])
baseline1765ch4

In [ ]:
obs_n2o = np.array([270.1, 272.1, 278.9])
obs_years = [1750, 1850, 1900]

def fit(x, a, b, c):
    return a + b*np.exp(c*(x-1750))

p, _ = scipy.optimize.curve_fit(fit, obs_years, obs_n2o, p0=[278.3, 1.45, 0.0171])
pl.plot(np.arange(1750, 1900), p[0]+p[1]*np.exp(p[2] * (np.arange(1750, 1900)-1750)))
pl.scatter(obs_years, obs_n2o, color='k')

baseline1765n2o = fit(1765, p[0], p[1], p[2])
baseline1765n2o

In [ ]:
species_index = {}
for i in range(len(species_to_include)):
    species_index[species_to_include[i]] = i

In [ ]:
species_index

## Generate 1.5 million ensemble members in batches of 1000

## Fill in the Configs

- Grab ClimateResponse configs from pre-calculated ensemble
- SpeciesConfigs will require careful handling

In [ ]:
seedgen = 1355763
run_config = RunConfig(ch4_lifetime_method=CH4LifetimeMethod.AERCHEMMIP)

# for all except temperature, the full time series is not important.
temp_out = np.ones((251, samples)) * np.nan
ohc_out = np.ones((samples)) * np.nan
fari_out = np.ones((samples)) * np.nan
faci_out = np.ones((samples)) * np.nan
co2_out = np.ones((samples)) * np.nan
fo3_out = np.ones((samples)) * np.nan

calibrated_f4co2_mean = df_cr['F_4xCO2'].mean()

for ibatch, batch_start in tqdm(enumerate(range(0, samples, batch_size)), total=samples/batch_size):
    batch_end = batch_start + batch_size
    configs = []
    for iconf in range(batch_start, batch_end):
        config_name = f"{iconf}"

        # Climate response configs
        climate_response = ClimateResponse(
            ocean_heat_capacity = df_cr.loc[iconf, 'c1':'c3'].values.squeeze(),
            ocean_heat_transfer = df_cr.loc[iconf, 'kappa1':'kappa3'].values.squeeze(),
            deep_ocean_efficacy = df_cr.loc[iconf, 'epsilon'],
            gamma_autocorrelation = df_cr.loc[iconf, 'gamma'],
            sigma_eta = df_cr.loc[iconf, 'sigma_eta'],
            sigma_xi = df_cr.loc[iconf, 'sigma_xi'],
            stochastic_run = True,
            seed = seedgen
        )
        seedgen = seedgen+399

        # Species configs
        # get defaults
        species_config = [copy.copy(species_config_from_default(species)) for species in species_to_include]

        # CO2 carbon cycle sensitivity and pre-industrial concentrations
        species_config[40].iirf_0 = df_cc.loc[iconf, 'r0']
        species_config[40].iirf_airborne = df_cc.loc[iconf, 'rA']
        species_config[40].iirf_cumulative = df_cc.loc[iconf, 'rC']
        species_config[40].iirf_temperature = df_cc.loc[iconf, 'rT']
        species_config[40].baseline_concentration = df_1765co2.loc[iconf, 'co2_concentration']
        
        # aerosol indirect params
        species_config[42].aci_params = {
            "scale": df_aci.loc[iconf, "beta"],
            "Sulfur": df_aci.loc[iconf, "shape_so2"], 
            "BC+OC": df_aci.loc[iconf, "shape_bcoc"]
        }

        # methane lifetime params (not varied, defaults are baked in)
        species_config[2].lifetime = 10.788405534387858
        species_config[2].natural_emissions_adjustment = 19.019783117809567
        species_config[2].soil_lifetime = 185

        # aerosol direct params
        for index in df_ari:
            try:
                species_config[species_index[index]].erfari_radiative_efficiency = df_ari.loc[iconf, index]
            except:
                pass

        # forcing scalings!
        for index in df_scaling:
            if index in ['minorGHG', 'solar_amplitude', 'solar_trend', 'CO2', 'contrails']:
                continue
            species_config[species_index[index]].scale = df_scaling.loc[iconf, index]
        species_config[38].scale = df_scaling.loc[iconf, 'solar_amplitude']
        for idx in range(11, 38):
            species_config[idx].scale = df_scaling.loc[iconf, 'minorGHG']

        # overwrite CO2 forcing scaling with correlated parameter from lambda1
        species_config[40].scale = 1+ 0.561*(calibrated_f4co2_mean - df_cr.loc[iconf,'F_4xCO2'])/calibrated_f4co2_mean
            
        # ozone scalings
        for index in df_ozone:
            if index=='ODS':
                for idx in range(11, 26):
                    species_config[idx].ozone_radiative_efficiency = df_ozone.loc[iconf, 'ODS']
            else:
                species_config[species_index[index]].ozone_radiative_efficiency = df_ozone.loc[iconf, index]
            
        # volcanic efficacy is something like 0.6. Could in future be sampled.
        species_config[39].efficacy = 0.6

        # rebase SLCFs and important reactive GHGs
        species_config[2].baseline_concentration = baseline1765ch4
        species_config[3].baseline_concentration = baseline1765n2o
        species_config[4].baseline_emissions = 2
        species_config[5].baseline_emissions = 1.2
        species_config[6].baseline_emissions = 10
        species_config[7].baseline_emissions = 4
        species_config[8].baseline_emissions = 46/14 * 2
        species_config[9].baseline_emissions = 10
        species_config[10].baseline_emissions = 174
        
        # normalisations for methane lifetime
        species_config[2].eesc_normalisation = 1069.9435837162791
        species_config[2].normalisation = 989.2548431552511
        species_config[3].normalisation = 50.32527327968029
        species_config[8].normalisation = 103.18436508571429
        species_config[9].normalisation = 133.940902
        species_config[10].normalisation = 632.2634499999999
        species_config[11].normalisation = 222.3461292077625
        species_config[12].normalisation = 509.1982990342465
        species_config[13].normalisation = 71.23047896621004
        species_config[14].normalisation = 16.35884729018265
        species_config[15].normalisation = 9.1098520186758
        species_config[16].normalisation = 223.5300098436073
        species_config[17].normalisation = 22.09069583538813
        species_config[18].normalisation = 21.774923623287677
        species_config[19].normalisation = 77.50162618858448
        species_config[20].normalisation = 58.028424414383494
        species_config[21].normalisation = 5.266147569189498
        species_config[22].normalisation = 1.3226580500342493
        species_config[23].normalisation = 3.798374400399544
        species_config[24].normalisation = 3.059724184771689
        species_config[25].normalisation = 0.2707454296757991
        configs.append(Config(config_name, climate_response, species_config))
        

        
    fair = FAIR(scenarios, configs, run_config=run_config)
    fair.run()
    
    # at this point dump out some batch output and put the constraining in another sheet
    temp_out[:, batch_start:batch_end] = fair.temperature[85:, 0, :, 0, 0]
    fair.calculate_ocean_heat_content_change()
    ohc_out[batch_start:batch_end] = fair.ocean_heat_content_change[253, 0, :, 0, 0]-fair.ocean_heat_content_change[206, 0, :, 0, 0]
    co2_out[batch_start:batch_end] = fair.concentration_array[249, 0, :, 40, 0]
    fari_out[batch_start:batch_end] = fair.forcing_array[240:250, 0, :, 41, 0].mean(axis=0)
    faci_out[batch_start:batch_end] = fair.forcing_array[240:250, 0, :, 42, 0].mean(axis=0)
    fo3_out[batch_start:batch_end] = fair.forcing_array[254, 0, :, 43, 0]

In [ ]:
# use F2xCO2 and scale factor to determine ECS: this is what each climate simulation actually "sees"
erf_2co2 = 3.934168323890023
calibrated_f4co2_mean = df_cr['F_4xCO2'].mean()

ecs = np.ones(samples) * np.nan
tcr = np.ones(samples) * np.nan
for iconf in tqdm(range(samples)):
    ebm = EnergyBalanceModel(
        ocean_heat_capacity = np.array([df_cr.loc[iconf,'c1'], df_cr.loc[iconf,'c2'], df_cr.loc[iconf, 'c3']]),
        ocean_heat_transfer = np.array([df_cr.loc[iconf,'kappa1'], df_cr.loc[iconf,'kappa2'], df_cr.loc[iconf,'kappa3']]),
        deep_ocean_efficacy = df_cr.loc[iconf,'epsilon'],
        forcing_4co2 = 2*erf_2co2 * (1+ 0.561*(calibrated_f4co2_mean - df_cr.loc[iconf,'F_4xCO2'])/calibrated_f4co2_mean) #2*erf_2co2*df_scale.loc[i,'CO2'],
    )
    ebm.emergent_parameters()
    ecs[iconf] = ebm.ecs
    tcr[iconf] = ebm.tcr

In [ ]:
np.save('../data/ar6_ensemble_batches_rcp/temperature_1850-2100.npy', temp_out, allow_pickle=True)
np.save('../data/ar6_ensemble_batches_rcp/ohc_2018_minus_1971.npy', ohc_out, allow_pickle=True)
np.save('../data/ar6_ensemble_batches_rcp/co2_2014.npy', co2_out, allow_pickle=True)
np.save('../data/ar6_ensemble_batches_rcp/fari_2005-2014_mean.npy', fari_out, allow_pickle=True)
np.save('../data/ar6_ensemble_batches_rcp/faci_2005-2014_mean.npy', faci_out, allow_pickle=True)
np.save('../data/ar6_ensemble_batches_rcp/fo3_2019.npy', fo3_out, allow_pickle=True)
np.save('../data/ar6_ensemble_batches_rcp/ecs.npy', ecs, allow_pickle=True)
np.save('../data/ar6_ensemble_batches_rcp/tcr.npy', tcr, allow_pickle=True)

In [ ]:
pl.hist(fo3_out)
np.percentile(fo3_out, (5, 50, 95))